In [19]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `llama 1b` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Load tokenizer and model from HuggingFace
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

# 📥 User Input: Enter Document Text
document = input("Enter your document text for paraphrasing:\n")

# 📊 Split the document into smaller chunks
def split_into_chunks(text, max_tokens=750):
    tokens = tokenizer(text, return_tensors="pt", truncation=False).input_ids[0]
    chunks = []
    for i in range(0, len(tokens), max_tokens):
        chunk = tokens[i:i + max_tokens]
        chunks.append(tokenizer.decode(chunk, skip_special_tokens=True))
    return chunks

# 🛠️ Paraphrase each chunk
paraphrased_chunks = []
chunks = split_into_chunks(document)

for i, chunk in enumerate(chunks):
    # Add a context transition if not the first chunk
    if i > 0:
        chunk = chunks[i-1][-100:] + "\n" + chunk  # Add previous chunk's tail for context

    prompt = (
        "You are an AI assistant trained for paraphrasing.\n"
        f"Original Text:\n{chunk}\n"
        "Paraphrased Text:"
    )

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=600)
    output = model.generate(
        **inputs,
        max_length=1000,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )

    paraphrased = tokenizer.decode(output[0], skip_special_tokens=True)
    paraphrased = paraphrased.split("Paraphrased Text:")[-1].strip()
    paraphrased_chunks.append(paraphrased)

# 📝 Combine the paraphrased chunks into one document
paraphrased_document = "\n".join(paraphrased_chunks)

# 📤 Display paraphrased document
print("\n--- Paraphrased Document ---\n")
print(paraphrased_document)

Enter your document text for paraphrasing:
There are beautiful things above and roundabout them; and if they gradually grow to feel that the whole world is nothing but muck, their power of usefulness is gone. If the whole picture is painted black there remains no hue whereby to single out the rascals for distinction from their fellows.  

--- Paraphrased Document ---

There are amazing things in the world; however, as they grow more and more, they start to lose their usefulness. If everything is painted black, it's difficult to distinguish between good and bad people. The beauty and wonder of the world are overshadowed by the dirt and ugliness that surrounds us.


In [1]:
# Install required packages
!pip install transformers streamlit ngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.4 MB/s eta 0:00:00


In [2]:
%%writefile app.py
import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the tokenizer and model
@st.cache_resource
def load_model():
    tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
    model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
    return tokenizer, model

tokenizer, model = load_model()

# Streamlit App
st.title("📝 Paraphrasing Assistant with LLaMA 🤖")

user_input = st.text_area("Enter your text to paraphrase:", height=200)

if st.button("Paraphrase"):
    if user_input:
        with st.spinner("Paraphrasing..."):
            prompt = (
                "You are an AI assistant skilled in paraphrasing. "
                "Your task is to rephrase the given text while preserving its original style, tone, and nuance.\n"
                f"Original Text:\n{user_input}\n"
                "Paraphrased Text:"
            )
            inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=600)
            output = model.generate(
                **inputs,
                max_length=600,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                pad_token_id=tokenizer.eos_token_id
            )
            paraphrased = tokenizer.decode(output[0], skip_special_tokens=True)
            paraphrased = paraphrased.split("Paraphrased Text:")[-1].strip()

            st.success("Paraphrasing Complete!")
            st.write("### Paraphrased Text:")
            st.write(paraphrased)
    else:
        st.warning("Please enter text to paraphrase.")

st.markdown("---")
st.markdown("Built using **Streamlit** and **LLaMA**.")


Writing app.py


In [10]:
# Start Streamlit app in the background
!streamlit run app.py &>/dev/null &


In [5]:
# Install pyngrok using pip
!pip install pyngrok

In [7]:
# Add your ngrok authentication token
!ngrok config add-authtoken 2qpruL9I6szooqm9VpbJxQ0st1c_6XFijP2MQJRMW5PqRFzXq


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [12]:
# Start Streamlit app in the background
!streamlit run app.py &>/dev/null &


In [14]:
# Kill any existing ngrok processes
!pkill -f ngrok


In [15]:
# Clear ngrok configuration
!rm -rf /root/.ngrok2


In [16]:
# Re-authenticate with your ngrok token
!ngrok config add-authtoken 2qpruL9I6szooqm9VpbJxQ0st1c_6XFijP2MQJRMW5PqRFzXq


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [17]:
# Start Streamlit app
!streamlit run app.py &>/dev/null &


In [ ]:
# Install necessary libraries
!pip install transformers streamlit ngrok pyngrok --quiet

# Start Streamlit on port 8501
!streamlit run app.py --server.port=8501 --server.address=0.0.0.0 &>/dev/null &

# Start ngrok tunnel
!ngrok http 8501 --log=stdout


INFO[12-28|07:32:29] no configuration paths supplied 
INFO[12-28|07:32:29] using configuration at default config path path=/root/.config/ngrok/ngrok.yml
INFO[12-28|07:32:29] open config file                         path=/root/.config/ngrok/ngrok.yml err=nil
t=2024-12-28T07:32:30+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]
t=2024-12-28T07:32:30+0000 lvl=info msg="client session established" obj=tunnels.session
t=2024-12-28T07:32:30+0000 lvl=info msg="tunnel session started" obj=tunnels.session
t=2024-12-28T07:32:30+0000 lvl=info msg="started tunnel" obj=tunnels name=command_line addr=http://localhost:8501 url=https://4b7d-34-125-2-185.ngrok-free.app
t=2024-12-28T07:32:37+0000 lvl=info msg="join connections" obj=join id=6ff82c1a525c l=127.0.0.1:8501 r=41.232.44.90:59979
t=2024-12-28T07:32:37+0000 lvl=info msg="join connections" obj=join id=ab02880453dc l=127.0.0.1:8501 r=41.232.44.90:59979
t=2024-12-28T07:32:37+0000 lvl=info msg="join connections" 